In [8]:
from huggingface_hub import hf_hub_download
import re
from PIL import Image

from transformers import NougatProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch

processor = NougatProcessor.from_pretrained("facebook/nougat-base")
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-base")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
# prepare PDF image for the model
filepath = "IMG_8044.png"
image = Image.open(filepath)
pixel_values = processor(image, return_tensors="pt").pixel_values

# generate transcription (here we only generate 30 tokens)
outputs = model.generate(
    pixel_values.to(device),
    min_length=1,
    max_new_tokens=500,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
)

sequence = processor.batch_decode(outputs, skip_special_tokens=True)[0]
sequence = processor.post_process_generation(sequence, fix_markdown=False)
# note: we're using repr here such for the sake of printing the \n characters, feel free to just print the sequence
repr(sequence)

"\n\n## 3 Mass and Weight\n\nIn SI, _kilograms_ are used for mass and _neutrons_ for weight (force). The units are different, and there is no confusion between the variables. However, for years the term _pound_ has been used for both mass and weight. This usage has obscured the distinction between the two: mass is a constant property of an object; weight varies with the gravitational field. Even the conventional use of the abbreviations _lm_ and _lf_(_to_distinguish between pounds-mass and pounds-force) has not helped eliminate the confusion.\n\nIt is true that an object with a mass of one pound will have an earthly weight of one pound, but this is true only on the earth. The weight of the same object will be much less on the moon. Therefore, care must be taken when working with mass and force in the same problem.\n\nThe relationship that converts mass to weight is familiar to every engineering student.\n\n\\[W = m_{\\mathit{\\mathit{\\mathit{\\mathit{\\mathit{\\mathit{\\mathit{\\mathi